In [60]:
#Data cleanup here

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import csv
import json
import requests
import seaborn as sns
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
import geopandas as gpd
import time

In [43]:
# Pull csv using URL of dataset uploaded to repo

hubs = ['NY', 'CA', 'DC', 'MA', 'MO', 'MI', 'WV', 'AL', 'IN']

types = ['1Bedroom', 
         '2Bedroom', 
         '3Bedroom',
         '4Bedroom', 
         '5BedroomOrMore', 
         'Sfr', 
         'Studio']

df_list = []

for d in types:
    path = f'zillow-house-price-data/City_MedianRentalPrice_{d}.csv'
    dataframe = pd.read_csv(path, error_bad_lines=False)
    dataframe = dataframe[['State', 'RegionName','CountyName','2015-01', 
                            '2015-02','2015-03','2015-04','2015-05', 
                            '2015-06','2015-07','2015-08','2015-09',
                            '2015-10','2015-11','2015-12','2016-01', 
                            '2016-02','2016-03','2016-04','2016-05',
                            '2016-06','2016-07','2016-08','2016-09', 
                            '2016-10','2016-11','2016-12','2017-01', 
                            '2017-02','2017-03','2017-04','2017-05',
                            '2017-06','2017-07','2017-08','2017-09',
                            '2017-10','2017-11','2017-12','2018-01',
                            '2018-02','2018-03','2018-04','2018-05',
                            '2018-06','2018-07','2018-08','2018-09',
                            '2018-10','2018-11','2018-12']]
    dataframe = dataframe.loc[dataframe['State'].isin(hubs)]
    # Convert the data to long form
    dataframe = pd.melt(dataframe, id_vars=['State','RegionName', 'CountyName'])

    #Rename columns
    dataframe.columns = ['State', 'RegionName', 'CountyName', 'Time', 'Value']

    dataframe['Time'] = pd.to_datetime(dataframe['Time']) 
    dataframe['Year'] = pd.DatetimeIndex(dataframe['Time']).year
    dataframe['Month'] = pd.DatetimeIndex(dataframe['Time']).month
    dataframe['Type'] = d
    
    dataframe = dataframe.loc[dataframe['Time'].isin(['2015', '2016', '2017', '2018'])]
    df_list.append(dataframe)


In [59]:
merged_df = df_list[0]

for number in np.arange(1,6):
    merged_df = merged_df.append(df_list[number])


merged_df.head()

State    RegionName          CountyName       Time   Value  Year  Month  \
0    NY      New York       Queens County 2015-01-01  2200.0  2015      1   
1    CA   Los Angeles  Los Angeles County 2015-01-01  1755.0  2015      1   
2    CA     San Diego    San Diego County 2015-01-01  1690.0  2015      1   
3    CA      San Jose  Santa Clara County 2015-01-01  2335.0  2015      1   
4    IN  Indianapolis       Marion County 2015-01-01   566.0  2015      1   

       Type  
0  1Bedroom  
1  1Bedroom  
2  1Bedroom  
3  1Bedroom  
4  1Bedroom

In [58]:
df_avg = merged_df.dropna()
df_avg = df_avg.groupby(['State', 'CountyName'])['Value'].mean()
df_avg = df_avg.reset_index()
df_avg

State          CountyName        Value
0      AL      Baldwin County  1368.636364
1      AL       Coffee County   896.250000
2      AL    Jefferson County   982.961538
3      AL      Madison County  1126.916667
4      AL       Mobile County   893.625000
..    ...                 ...          ...
108    NY  Westchester County  2272.686047
109    WV     Berkeley County  1131.666667
110    WV       Cabell County   736.250000
111    WV      Kanawha County   723.500000
112    WV   Monongalia County  1015.500000

[113 rows x 3 columns]

In [66]:
counties_usa = gpd.read_file('cb_2018_us_county_20m/cb_2018_us_county_20m.shp')
counties_usa.columns = ['STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'GEOID', 'CountyName', 'LSAD',
       'ALAND', 'AWATER', 'geometry']
counties_usa.head()

STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID CountyName LSAD  \
0      37      017  01026336  0500000US37017  37017     Bladen   06   
1      37      167  01025844  0500000US37167  37167     Stanly   06   
2      39      153  01074088  0500000US39153  39153     Summit   06   
3      42      113  01213687  0500000US42113  42113   Sullivan   06   
4      48      459  01384015  0500000US48459  48459     Upshur   06   

        ALAND    AWATER                                           geometry  
0  2265887723  33010866  POLYGON ((-78.90200 34.83527, -78.79960 34.850...  
1  1023370459  25242751  POLYGON ((-80.49737 35.20210, -80.29542 35.502...  
2  1069181981  18958267  POLYGON ((-81.68699 41.13596, -81.68495 41.277...  
3  1165338428   6617028  POLYGON ((-76.81373 41.59003, -76.22014 41.541...  
4  1509910100  24878888  POLYGON ((-95.15274 32.66095, -95.15211 32.902...

In [ ]:
df_county = pd.merge(counties_usa, df_avg, on="CountyName", suffixes=("_shape", ""))

df_county

In [ ]:
geosource_2015 = GeoJSONDataSource(geojson = df_county.to_json())


In [ ]:
## Plot Median Rent by County
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 4000) #make sure you set the low and high, if you forget it won't map
# Define custom tick labels for color bar.
tick_labels = {'0': '0', '500': '500',
 '1000':'1000', '1500':'1500',
 '2000':'2000', '2500':'2500',
 '3000':'3000', '3500':'3500',
 '4000':'4000+'}
# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)
# Create figure object.
p_2015 = figure(title = 'Median Rental Prices, 2015', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p_2015.xgrid.grid_line_color = None
p_2015.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p_2015.patches('xs','ys', source = geosource_2015, #your geojson file
                   fill_color = {'field' :'Value', #this has to be your column name
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p_2015.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@RegionName'),
                               ('Median Rent','@Value{int}')])) #if you add {int} then it writes as integer, otherwise scientific notation
                                                                #the @ shows the column name, so here I am saying "in hover box show "Median Rent: [value in 'Value' column]{in integer format}
# Specify layout
p.add_layout(color_bar, 'below')
show(p_2015)